## Simulate New Control System
This notebook serves as a tutorial for building a simulation for an arbitrary system whose model is defined by a stochastic differential equation (SDE). 

```CBFkit``` provides functionality for simulating arbitrary controlled dynamical systems given that they may be modelled by SDEs or ordinary differential equations (ODEs). As such, this tutorial will walk the user through the process of generating code for simulating new models. It will then instruct the user on how to set up the remaining items required for simulating the controlled dynamical system, and conclude by executing the simulation and logging the results for analysis.

Accordingly, this notebook is organized into 3 sections:
1. Model Generation: generate the code required to model and control the system dynamics for simulation
2. Simulation Setup: configure the dynamics, controller, state estimator, etc. for simulation
3. Execution and Data Logging: execute the simulation and save the data for analysis

Before proceeding, we need some generic ```jax``` imports.

In [1]:
from jax import Array, jit
import jax.numpy as jnp

### 1. Model Generation

This section provides instruction on how to generate new code for simulating the controlled (or uncontrolled) dynamical system of the user's choice. 

The main heavy lifting will be done by the ```generate_model``` function found in the ```systems.create_new_system``` module.

In [2]:
from cbfkit.codegen.create_new_system import generate_model

Next, the system model must be specified. Importantly, we currently support systems of the form $\dot x = f(x) + g(x)u + d(x)$, where $x \in \mathbb R^n$ is the state, $u \in \mathbb R^m$ the control, and where $f: \mathbb R^n \mapsto \mathbb R^n$ represents the drift dynamics, $g: \mathbb R^n \mapsto \mathbb R^{n \times m}$ the control matrix, and $d: \mathbb R^n \mapsto \mathcal D \subseteq \mathbb R^n$ represents some disturbance (or term representing the diffusion term in a SDE).

In addition, we support parametric definitions of these functions in the sense that they may be defined with respect to a set of static parameters $\theta \in \Theta \subseteq \mathbb R^p$, which are specified a priori, i.e., for $\dot x = f(x, \theta) + g(x, \theta)u + d(x).$

<span style="color:red">

In this tutorial, we will be working with two single integrators. Their states are concetenated in a single vector as
$x = [x_1, x_2, x_3, x_4]$ where $x_1, x_2$ are X, Y coordinate positions of first robot and $x_3,x_4$ are positions of the second robot. The single integrator for each robot is modeled as follows: $\begin{align}\dot x_1 &= u_1 \nonumber \\ \dot x_2 &=  u_2 \nonumber\end{align}$

</span>


As such, we define these relevant terms symbolically with strings as follows (omitting $d(x)$, which does not go in the code-gen):

<span style="color:red">

- drift dynamics has to be 1D array

- control matrix has to be 2D array
</span>

In [3]:
# NEW: Drift dynamics has to be 1D array
drift_dynamics = " [0, 0, 0, 0] "

# NEW: Control matrix goes as 2D array
control_matrix = "[[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]"

Because we are generating code, we must specify a location for this code to be saved.

In [4]:
target_directory = "./tutorials"
model_name = "two_augmented_single_integrators"

The dynamics does not have any parameter but it is possible to pass a parameter too. See other examples to see how. Here we initialize empty parameter for dynamics as follows

```params = {"dynamics": {"variable_name: data_type": value}}```

In [5]:
params = {"dynamics": {}}

We are now able to call ```generate_model``` to generate the dynamics code.

In [6]:
generate_model.generate_model(
    directory=target_directory,
    model_name=model_name,
    drift_dynamics=drift_dynamics,
    control_matrix=control_matrix,
    params=params,
)

Generated ROS2 controller node script at ./tutorials/two_augmented_single_integrators/ros2/controller.py
Generated ROS2 sensor node script at ./tutorials/two_augmented_single_integrators/ros2/sensor.py
Generated ROS2 estimator node script at ./tutorials/two_augmented_single_integrators/ros2/estimator.py
Generated ROS2 plant node script at ./tutorials/two_augmented_single_integrators/ros2/plant_model.py
Generated configuration script at ./tutorials/two_augmented_single_integrators/ros2/config.py
Generated bash script to run ROS2 nodes at: ./tutorials/two_augmented_single_integrators/run_ros2_nodes.sh


(4, 4)

As you can see, a directory called ```tutorials``` has been created in the current working directory. It contains a multitude of new files, a breakdown of which may be seen below:

#### Adding a Control Law
Having defined our control-affine dynamics model, it is now time to define an arbitrary (nominal) controller. The ```CBFkit``` supports defining control laws that are a function of the state, time, and static parameters. For example, if we wanted to implement the simple proportional control law for each robot $u(x) = -k_p (x-goal)$, then we would define the ```nominal_control_law``` variable as follows.

In [7]:
goal1_x = 1.0
goal1_y = 1.0
goal2_x = -1.0
goal2_y = -1.0
nominal_control_law = " [ -k_p * (x[0]-goal1_x), -k_p * (x[1]-goal1_y), -k_p * (x[2]-goal2_x), -k_p * (x[3]-goal2_y) ] " 
params["controller"] = {"goal1_x: float": goal1_x, "goal1_y: float": goal1_y, "goal2_x: float": goal2_x, "goal2_y: float": goal2_y, "k_p: float": 1.0} 

<span style="color:red">
Note that goal coordinates can also be passed as a asingle vector parameter
</span>

We may then generate the model and controller code using ```generate_model``` as follows.

In [8]:
generate_model.generate_model(
    directory=target_directory,
    model_name=model_name,
    drift_dynamics=drift_dynamics,
    control_matrix=control_matrix,
    nominal_controller=nominal_control_law,
    params=params,
)

Generated ROS2 controller node script at ./tutorials/two_augmented_single_integrators/ros2/controller.py
Generated ROS2 sensor node script at ./tutorials/two_augmented_single_integrators/ros2/sensor.py
Generated ROS2 estimator node script at ./tutorials/two_augmented_single_integrators/ros2/estimator.py
Generated ROS2 plant node script at ./tutorials/two_augmented_single_integrators/ros2/plant_model.py
Generated configuration script at ./tutorials/two_augmented_single_integrators/ros2/config.py
Generated bash script to run ROS2 nodes at: ./tutorials/two_augmented_single_integrators/run_ros2_nodes.sh


(4, 4)

The file hierarchy now looks like the following, with a new ```controllers``` folder containing ```___init___.py```, ```controller_1.py``` (the specified nominal controller), and ```zero_controller.py``` (applying zero control inputs) files.

#### Adding Certificate Functions

Now, we have our dynamics model and we have our nominal control law. But what if we want our controller to obey some state and/or input constraints? 

We can specify constraint functions symbolically in a similar fashion to the dynamics. For example, if we want to enforce minimu m distance constraint $||(x_1 - x_2)||^2 \geq 1 $,we can use the constraint functions $h_1(x) = ||(x_1 - x_2)||^2 - 1$ and encode it in a CBF-QP controller. As such, we can define a variable ```state_constraint_funcs``` in the following way.

In [9]:
state_constraint_funcs = [
    "(x[0]-x[2])*(x[0]-x[2]) + (x[1]-x[3])*(x[1]-x[3]) - 1"
] 

We can encode control Lyapunov functions in a similar way. For example, if our candidate CLF is $V(x) = x_1^2 + x_2^2 -1$, then we can define the variable ```lyapunov_functions``` as follows.

In [10]:
lyapunov_functions = " (x[0]-goal1_x)**2 + (x[1]-goal1_y)**2 + (x[2]-goal2_x) + (x[3]-goal2_y)**2"  # "x[0]**2 + x[1]**2 - radius"
params["clf"] = [
    {
        "goal1_x: float": goal1_x,
        "goal1_y: float": goal1_y,
        "goal2_x: float": goal2_x,
        "goal2_y: float": goal2_y,
    }
]

We can now generate our full model code with the ```generate_model``` module as follows.

In [11]:
generate_model.generate_model(
    directory=target_directory,
    model_name=model_name,
    drift_dynamics=drift_dynamics,
    control_matrix=control_matrix,
    barrier_funcs=state_constraint_funcs,
    lyapunov_funcs=lyapunov_functions,
    nominal_controller=nominal_control_law,
    params=params,
)

Generated ROS2 controller node script at ./tutorials/two_augmented_single_integrators/ros2/controller.py
Generated ROS2 sensor node script at ./tutorials/two_augmented_single_integrators/ros2/sensor.py
Generated ROS2 estimator node script at ./tutorials/two_augmented_single_integrators/ros2/estimator.py
Generated ROS2 plant node script at ./tutorials/two_augmented_single_integrators/ros2/plant_model.py
Generated configuration script at ./tutorials/two_augmented_single_integrators/ros2/config.py
Generated bash script to run ROS2 nodes at: ./tutorials/two_augmented_single_integrators/run_ros2_nodes.sh


(4, 4)

This generates the following file hierarchy, which in addition now possesses the folder ```certificate_functions``` containing subfolders ```barrier_functions``` and ```lyapunov_functions``` with the specified barrier and lyapunov candidate functions.

In the ensuing section, it will be shown how these newly generated files may be used for simulating the controlled Van der Pol system.

### 2. Simulation Setup

The simulation will be executed by the ```execute``` function from the ```simulation.simulator``` module. In viewing the ```execute``` docs, we see that it expects the following arguments:

```
Args:
    x0 (State): initial (ground truth) state of the system
    dynamics (DynamicsCallable): specifies system dynamics
    sensor (SensorCallable): function for sensing the (complete or partial) state
    controller (ControllerCallable): function for computing the control input u
    estimator (EstimatorCallable): function for estimating the state x
    integrator (IntegratorCallable): function for numerically integrating the state forward in time
    dt (Time): length of simulation timestep (sec)
    num_steps (int): total number of timesteps in simulation. final time = num_steps * dt
    filepath (Optional[str], optional): location to save file. Defaults to None (no save).
```

Therefore, we need to define the correct objects for ```dynamics```, ```sensor```, ```controller```, ```estimator```, ```integrator```, ```dt```, and ```num_steps```, which may be done using the code we just generated. But first, we need some import statements.

In [12]:

# Provides access to execute (sim.execute)
import cbfkit.simulation.simulator as sim

# Access to CBF-CLF-QP control law
import cbfkit.controllers_and_planners.model_based.cbf_clf_controllers as cbf_clf_controllers

# Necessary housekeeping for using multiple CBFs/CLFs
from cbfkit.controllers_and_planners.model_based.cbf_clf_controllers.utils.certificate_packager import (
    concatenate_certificates,
)

# Suite of zeroing barrier function derivative conditions (forms of Class K functions)
from cbfkit.controllers_and_planners.model_based.cbf_clf_controllers.utils.barrier_conditions import (
    zeroing_barriers,
)

# Exponentially stable derivative condition for CLF
from cbfkit.controllers_and_planners.model_based.cbf_clf_controllers.utils.lyapunov_conditions.exponential_stability import (
    e_s,
)

# Assuming perfect, complete state information
from cbfkit.sensors import perfect as sensor

# With perfect sensing, we can use a naive estimate of the state
from cbfkit.estimators import naive as estimator

# Use forward-Euler numerical integration scheme
from cbfkit.utils.numerical_integration import forward_euler as integrator

Now, import the newly generated dynamics module.

In [13]:
from tutorials import two_augmented_single_integrators

With that, it is time to define simulation parameters and instantiate the objects imported for simulation purposes.

In [14]:
# Simulation Parameters
SAVE_FILE = f"tutorials/{model_name}/simulation_data"  # automatically uses .csv format
DT = 0.1  # 1e-2
TF = 10  # 4  # 10.0
N_STEPS = int(TF / DT) + 1
INITIAL_STATE = jnp.array([-0.9, -0.9, 0.9, 0.9])
ACTUATION_LIMITS = jnp.array(
    [100.0, 100.0, 100.0, 100.0]
)  # Box control input constraint, i.e., -100 <= u <= 100

<span style="color:red">
Note that ACTUATION_LIMITS should be defined for every control input. Since we have 4 inputs (2 for each single integrator robot), ACTUATION_LIMITS should have 4 elements.
</span>

In [15]:
# Dynamics function 
dynamics = two_augmented_single_integrators.plant() #epsilon=eps, perturbation=generate_stochastic_perturbation(sigma, DT))

# Create barrier functions with linear class K function derivative conditions
b1 = two_augmented_single_integrators.certificate_functions.barrier_functions.cbf1_package(
    certificate_conditions=zeroing_barriers.linear_class_k(alpha=1.0),
)
barriers = concatenate_certificates(b1)

# Create lyapunov function with exponential stability derivative condition
l1 = two_augmented_single_integrators.certificate_functions.lyapunov_functions.clf1_package(
    certificate_conditions=e_s(c=2.0),
    goal1_x=goal1_x,
    goal1_y=goal1_y,
    goal2_x=goal2_x,
    goal2_y=goal2_y,
)
lyapunov = concatenate_certificates(l1)

# Instantiate nominal controller
nominal_controller = two_augmented_single_integrators.controllers.controller_1(
    goal1_x=goal1_x, goal1_y=goal1_y, goal2_x=goal2_x, goal2_y=goal2_y, k_p=1
)

# Instantiate CBF-CLF-QP control law
cbf_clf_controller = cbf_clf_controllers.vanilla_cbf_clf_qp_controller(
    control_limits=ACTUATION_LIMITS,
    nominal_input=nominal_controller,
    dynamics_func=dynamics,
    barriers=barriers,
    lyapunovs=lyapunov,
    relaxable_clf=True,
)

### 3. Simulation Execution

Now we are ready to execute the simulation with ```sim.execute```.

In [16]:
x, _u, _z, _p, dkeys, dvalues = sim.execute(
    x0=INITIAL_STATE,
    dt=DT,
    num_steps=N_STEPS,
    dynamics=dynamics,
    integrator=integrator,
    controller=cbf_clf_controller,
    sensor=sensor,
    estimator=estimator, #perturbation=generate_stochastic_perturbation(sigma=sigma, dt=DT),
    filepath=SAVE_FILE,
)

100%|██████████| 101/101 [00:00<00:00, 110.44it/s]
